# Tract🩻Segm: Fake🏴‍☠️predictions

clarification on https://www.kaggle.com/competitions/uw-madison-gi-tract-image-segmentation/discussion/320908

In [ ]:
import os, glob
import pandas as pd
import matplotlib.pyplot as plt

DATASET_FOLDER = "/kaggle/input/uw-madison-gi-tract-image-segmentation"

In [ ]:
df_train = pd.read_csv(os.path.join(DATASET_FOLDER, "train.csv"))
print(f"size: {len(df_train)}")
display(df_train.head())

In [ ]:
df_ssub = pd.read_csv(os.path.join(DATASET_FOLDER, "sample_submission.csv"))
print(f"size: {len(df_ssub)}")
display(df_ssub.head())

In [ ]:
ex_ids = [i not in df_ssub["id"] for i in df_train["id"]]
print(f"missing: {len(ex_ids)}")

## Find 🔎 all images

In [ ]:
# Switch between parsing test or train folder
PATTERN = ("case*", "case*_day*", "scans", "*.png")
if df_ssub.empty:
    print("using TRAIN dataset?")
    all_imgs = glob.glob(os.path.join(DATASET_FOLDER, "train", *PATTERN))
else:
    print("using TEST dataset!")
    all_imgs = glob.glob(os.path.join(DATASET_FOLDER, "test", *PATTERN))

print(f"images: {len(all_imgs)}")
labels = sorted(df_train["class"].unique())
print(f"labels: {labels}")

## Parse 🗄️ test dataset

In [ ]:
def extract_details(ip):
    img = os.path.basename(ip)
    im_name, _ = os.path.splitext(img)
    folders = ip.split(os.path.sep)
    case_day = folders[-3].split("_")
    case = case_day[0].replace("case", "")
    day = case_day[1].replace("day", "")
    slice_id = im_name.split("_")[1]
    img_path = img.replace(DATASET_FOLDER + "/", "")
    return {
        "id": f"case{case}_day{day}_slice_{slice_id}",
        "Case": int(case),
        "Day": int(day),
        "Slice": slice_id,
        "image": img,
        "image_path": img_path,
    }

extract_details(all_imgs[0])

In [ ]:
from tqdm.auto import tqdm

df = pd.DataFrame([extract_details(ip) for ip in tqdm(all_imgs)])
print(f"size: {len(df)}")
display(df.head(3))

In [ ]:
ex_ids = [i not in df_ssub["id"] for i in df["id"]]
print(f"missing: {len(ex_ids)}")

## 🏴‍ Dummy predictions

In [ ]:
sub = [{"id": row["id"], "class": lb, "predicted": f"{i} 1"}
       for _, row in tqdm(df.iterrows(), total=len(df)) for i, lb in enumerate(labels)]
df_sub = pd.DataFrame(sub)
display(df_sub.head())

### debugging the test format!!!

**seems the the hiddend key is the order which organizors were so nice and keep for themselves :/**

In [ ]:
keys_df = [f"{rr['id']}-//-{rr['class']}" for _, rr in tqdm(df_sub.iterrows())]
if not df_ssub.empty:
    keys_ss = [f"{rr['id']}-//-{rr['class']}" for _, rr in tqdm(df_ssub.iterrows())]
    assert len(df_sub) == len(df_ssub)
    missed_keys = [k for k in keys_df if k not in keys_ss]
    assert not missed_keys

### reorder snd submit...

In [ ]:
del df_ssub['predicted']
df_sub = df_ssub.merge(df_sub, on=['id','class'])

df_sub[['id', 'class', 'predicted']].to_csv("submission.csv", index=False)

!head submission.csv